# Experimentando com bancos de dados em Python

## Conectando com a base de dados

Existem sistemas de gerenciamento de bancos de dados que operam no mesmo processo da aplicação cliente - ou seja, são bibliotecas que você pode importar diretamente no seu programa. Por exemplo: a biblioteca ``sqlite3`` (https://docs.python.org/3/library/sqlite3.html) é parte do conjunto de bibliotecas padrão da linguagem Python, e permite criar bases de dados diretamente em arquivo ou memória, sem a necessidade de um programa em separado. Eis um exemplo de uso de sqlite3:

In [1]:
import sqlite3

db = sqlite3.connect("teste.db")
# Para criar a base em memória (base temporária) use o nome de arquivo
# ":memory:"

Note que o arquivo ``teste.db`` foi criado. 

In [2]:
import os

print("teste.db" in os.listdir())

True


Apesar do ``sqlite3`` criar o sistema de gerenciamento de banco de dados (SGBD) SQLite diretamente no nosso processo (neste caso, o kernel Python que está sendo executado por esse notebook), o objeto de acesso ainda é uma "conexão". Isso se deve ao fato de que SGBDs "normais" - aqueles que realmente usamos em produção - são geralmente *sistemas cliente-servidor*, onde o SGBD é um processo separado da aplicação cliente e a comunicação entre eles acontece através de uma conexão de rede.

<center><img src='imgs/client-server.png'></center>

**Atividade:**

Na figura acima, quem é o cliente do sistema de gerenciamento de banco de dados?

<div class="alert alert-success">

Sua resposta aqui! Dê dois cliques e edite.
    
    Escreva AQUI!

</div>

Uma conexão é simplesmente o conduíte entre o cliente e o SGBD. Em Python, a especificação PEP 249 (https://www.python.org/dev/peps/pep-0249/) define que uma conexão deve ter os métodos ``.close()`` (para fechar a conexão), ``.cursor()`` (para obter um *cursor*, que é o objeto para realmente interagir com o banco de dados), e o par ``.commit()``/``.rollback()`` (para gerenciamento de transações).

Vamos fechar essa conexão e trabalhar com o MySQL daqui em diante.

In [3]:
db.close()

In [4]:
os.remove("teste.db")

Antes de tudo vamos criar um banco de dados. Queremos criar um banco de dados para gerenciar as equipes do primeiro campeonato Insper de ninja-cowboy-urso!

- Ninja, com sua velocidade e furtividade, detona o cowboy.
- O cowboy atira no urso.
- O urso dá uma patada no ninja e acaba com ele.

Cada jogador tem um nome, uma equipe, e sua jogada favorita. As equipes tem um nome e um grito de guerra.

Portanto, nosso schema será:

```
jogador (id PK, nome_equipe, nome, preferencia)
equipe (nome PK, grito)
```

com a restrição:

```
jogador(nome_equipe) FK para equipe(nome)
```

E um dicionário de dados simples para este problema é:

<center><b>jogador</b></center>
<table>
    <tr>
        <th>Campo</th>
        <th>Tipo</th>
        <th>Significado</th>
        <th>Chave?</th>
        <th>Valores</th>
    </tr>
    <tr>
        <td>id</td>
        <td>int</td>
        <td>Identificador unico do jogador</td>
        <td>PK</td>
        <td>sem restrições</td>
    </tr>
    <tr>
        <td>nome_equipe</td>
        <td>string</td>
        <td>Nome da equipe na qual o jogador atua</td>
        <td>FK equipe(nome)</td>
        <td></td>
    </tr>
    <tr>
        <td>nome</td>
        <td>string</td>
        <td>Nome do jogador</td>
        <td></td>
        <td>Pode haver dois jogadores com mesmo nome. Máximo 80 caracteres.</td>
    </tr>
    <tr>
        <td>preferencia</td>
        <td>int</td>
        <td>Jogada preferida do jogador</td>
        <td></td>
        <td>0: Ninja, 1: Cowboy, 2: Urso</td>
    </tr>
</table>

<center><b>equipe</b></center>
<table>
    <tr>
        <th>Campo</th>
        <th>Tipo</th>
        <th>Significado</th>
        <th>Chave?</th>
        <th>Valores</th>
    </tr>
    <tr>
        <td>nome</td>
        <td>string</td>
        <td>Nome da equipe</td>
        <td>PK</td>
        <td>Máximo 30 caracteres</td>
    </tr>
    <tr>
        <td>grito</td>
        <td>string</td>
        <td>Grito de guerra da equipe</td>
        <td></td>
        <td>Maximo 80 caracteres</td>
    </tr>
</table>

**Atividade:** Escreva e rode o script de criação do banco de dados. Chame o banco de dados de ``torneio``. Compare com o gabarito em ``script_001.sql``.

In [5]:
!pip install mysql-connector-python


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\Vinicius Rodrigues\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Para conectar com o MySQL precisamos da biblioteca *mysql-connector-python*. 
Para instalar esta biblioteca:

**`pip install mysql-connector-python`**

In [6]:
from dotenv import load_dotenv
import insperautograder.jupyter as ia
import mysql.connector
import os

In [7]:
load_dotenv(override=True)

True

Vamos agora abrir uma conexão com o MySQL. Como agora temos realmente um sistema cliente-servidor, precisamos definir onde está o servidor, e precisamos também de informações de autenticação.

In [8]:
connection_options = {
    "host": "localhost",
    "user": "root",
    "password": "123456",
    "database": "torneio"
}

**Pergunta**: Faz sentido guardar o *user* e *password* dentro do próprio código fonte? Qual o jeito correto de lidar com username e password no seu produto?

<div class="alert alert-success">

Sua resposta aqui! Dê dois cliques e edite.
    
    Escreva AQUI!

</div>

**Resposta**: 

Não se deve guardar *user* e *password* dentro do código fonte, pois trata-se de:
- Informação sensível e que não deve ser acessada pelos programadores.
- Informação que muda com frequência - ou pelo menos deveria mudar!
- Dado puro, que deve ser separado do código.

Algumas alternativas populares para resolver esse problema são:
- Use um arquivo de configuração na conta que será usada para rodar a aplicação. Este arquivo deve ter todas as permissões de acesso negadas, exceto permissão de leitura para o próprio usuário da conta de execução da aplicação. O arquivo deverá ser lido então pela aplicação no momento da inicialização.
- Use uma variável de ambiente com essas credenciais. É a mesma coisa que no ítem anterior, só ao invés de ter um arquivo de configuração da aplicação, temos o arquivo de configuração da conta em si, com os mesmos requisitos em relação às restrições de permissão.

## Atualizando `.env`

### Tarefa:

Atualize o arquivo `.env`. Por enquanto ele provavelmente contém apenas as variáveis criadas para a API de correção automática:

```python
IAG_SERVER_URL="https://bigdata.insper-comp.com.br/iag"
IAG_SUBJECT="megadados"
IAG_OFFERING="xxxx"
IAG_TOKEN="iagtok_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
```

Vamos adicionar três novas variáveis:

- `MD_DB_SERVER`: o endereço para o host onde o mysql está instalado (provavelmente `"localhost"`)
- `MD_DB_SERVER`: o usuário para acesso ao banco (provavelmente `"root"`)
- `MD_DB_SERVER`: a senha do usuário.


O `.env` ficará neste formato (veja as três novas variáveis ao final):

```python
IAG_SERVER_URL="https://bigdata.insper-comp.com.br/iag"
IAG_SUBJECT="megadados"
IAG_OFFERING="xxxx"
IAG_TOKEN="iagtok_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
MD_DB_SERVER="localhost"
MD_DB_USERNAME="root"
MD_DB_PASSWORD="alguma_senha"
```

In [9]:
load_dotenv(override=True)

connection_options = {
    "host": os.getenv("MD_DB_SERVER"),
    "user": os.getenv("MD_DB_USERNAME"),
    "password": os.getenv("MD_DB_PASSWORD"),
    "database": "torneio"
}

**Abrindo a conexão:**

In [10]:
#A notação de duplo asterisco em Python permite usar um dicionário 
#no lugar de argumentos nomeados. A linha a ser executada abaixo equivale á:

connection = mysql.connector.connect(
    host=os.getenv("MD_DB_SERVER"),
    user=os.getenv("MD_DB_USERNAME"),
    password=os.getenv("MD_DB_PASSWORD"),
    database="torneio"
)

connection = mysql.connector.connect(**connection_options)

<div class="alert alert-info">

**IMPORTANTE:**

TODAS AS OPERAÇÕES REALIZADAS NESTA CONEXÃO SÃO TEMPORÁRIAS, ATÉ QUE O COMANDO `connection.commit()` seja executado!

</div>

<div class="alert alert-warning" role="alert">
    
**MAIS IMPORTANTE AINDA:**

ISSO NÃO VALE SE `AUTOCOMMIT` ESTIVER LIGADO!

</div>


Estudaremos melhor essa questão quando discutirmos transações em maiores detalhes.

Vamos criar alguns dados iniciais:

In [11]:
T1 = "Raposas Nerds"
T2 = "É nois"

NINJA = 0
COWBOY = 1
URSO = 2

times = {
    T1: "sudo vencer!",
    T2: "Olha eu mamãe!",
}

jogadores = [
    ("Raul Ayres", T1, NINJA),
    ("Luciano Hashimoto", T1, COWBOY),
    ("Rafael Montaigner", T1, URSO),
    ("Igor Miranda", T2, URSO),
    ("Andrew Ikeda", T2, COWBOY),
    ("Fábio Kurauchi", T2, NINJA),
]

Vamos inserir os dados dos times.

**Atividade:** Quem devemos inserir primeiro: dados na tabela `equipe` ou na tabela `jogador`?

<div class="alert alert-success">

Sua resposta aqui! Dê dois cliques e edite.
    
    Escreva AQUI!

</div>

Quando uma tabela tem chave estrangeira para outra tabela dizemos que ela é filha (*child*) da outra tabela, e que a outra tabela é a pai (*parent*).

Um *cursor* é o objeto que permite executar queries SQL e interagir com os resultados obtidos.

### Tentando inserir um jogador

Vamos tentar inserir um jogador:

In [12]:
try:
    with connection.cursor() as cursor:
        cursor.execute("INSERT INTO jogador (nome, nome_equipe, preferencia) VALUES ('Joao', 'sete e meia', 0)")
    connection.commit()
except mysql.connector.IntegrityError as exception:
    print(f"IntegrityError: {exception}")
    connection.rollback()

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`torneio`.`jogador`, CONSTRAINT `jogador_ibfk_1` FOREIGN KEY (`nome_equipe`) REFERENCES `equipe` (`nome`) ON DELETE CASCADE ON UPDATE CASCADE)


Perceba que obtemos uma exceção, pois estamos tentando utilizar uma equipe inexistente na tabela de `equipe`.

Isto responde nossa pergunta anterior: precisamos inserir primeiro as equipes. Vamos fazer isto utilizando um laço que fará todas as inserções:

In [13]:
try:
    with connection.cursor() as cursor:
        # Outra opção aqui seria o uso de cursor.executemany()
        for time, grito in times.items():
            cursor.execute("INSERT INTO equipe VALUES (%s, %s)", (time, grito))
    connection.commit()
except mysql.connector.IntegrityError as exception:
    print(f"IntegrityError: {exception}")
    connection.rollback()

IntegrityError: 1062 (23000): Duplicate entry 'Raposas Nerds' for key 'equipe.PRIMARY'


Note o comando ``commit`` acima: ele serve para efetivar a operação no banco de dados. Verifique no MySQL Workbench que a operação foi bem sucedida.

**Atividade:** Perceba que desta vez não especificamos os nomes das colunas no `INSERT`. É necessário deixar claro os campos da tabela?

<div class="alert alert-success">

Sua resposta aqui! Dê dois cliques e edite.
    
    Escreva AQUI!

</div>

Podemos tambem usar nossa conexão para verificar o estado da tabela ``equipe``:

In [14]:
with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM equipe")
    for item in cursor:
        print(item)

('É nois', 'Olha eu mamãe!')
('Raposas Nerds', 'sudo vencer --force!')


Podemos usar também nosso velho conhecido `pandas`!

In [15]:
import pandas as pd
pd.read_sql_query("SELECT * FROM equipe", connection)

ModuleNotFoundError: No module named 'pandas'

**Atividade:** Tente executar novamente a inserção de dados, o que acontece?

<div class="alert alert-success">

Sua resposta aqui! Dê dois cliques e edite.
    
    Escreva AQUI!

</div>

## Interlúdio: sanitização de entradas de dados

Por que usamos aqueles ``%s`` no comando a ser executado? Por que não simplesmente incluir diretamente os dados na *string* do comando SQL? Por exemplo:

```Python
cursor.execute("INSERT INTO equipe VALUES ('" + time + "', '" + grito + "')")
```

Muitas vezes o resultado é o mesmo! Contudo, e SE executássemos a query abaixo...

time = "Robert', ''); DROP DATABASE torneio; -- "
grito = ""

with connection.cursor() as cursor:
    for result in cursor.execute(
            "INSERT INTO equipe VALUES ('" + time + "', '" + grito + "')",
            multi=True,
    ):
        print(result)
    connection.commit()

with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM equipe")
    for item in cursor:
        print(item)

connection.close()

A célula acima está desabilitada. Mas se quiser executar passe para tipo "Code" e rode. Depois, garanta que a conexão está fechada com `connection.close()`, volte para o começo do notebook e rode tudo de novo exceto a célula mortal!

![Little Bobby Tables](imgs/exploits_of_a_mom.png)

[Exploits of a mom](https://xkcd.com/327/)


Existem jeitos de quebrar o sistema com esse tipo de ataque, onde o hacker tenta inserir código SQL mal-intencionado em formulários, na expectativa de que o programador não tenha feito a sanitização das entradas. Esse ataque chama-se ***'SQL injection'***.

Fim do interlúdio, voltamos à apresentação principal.

Vamos agora inserir os jogadores. Neste caso não precisamos inserir a chave primária, pois ela é gerada automaticamente. Consequentemente temos que especificar que estamos inserindo apenas as demais colunas - temos que nomear as colunas na qual estamos inserindo os dados.

In [16]:
try:
    with connection.cursor() as cursor:
        cursor.executemany(
            "INSERT INTO jogador (nome, nome_equipe, preferencia) VALUES (%s, %s, %s)",
            jogadores)
    connection.commit()
except Exception as e:
    print(e)
    connection.rollback()

Vamos verificar se funcionou. Para evitar ter criar os cursores a todo momento e coletar os resultados vamos construir uma classe auxiliar, só para simplificar o código:

In [17]:
class ConnectionHelper:

    def __init__(self, connection):
        self.connection = connection

    def __call__(self, query, args=None):
        with self.connection.cursor() as cursor:
            print("Executando query:")
            cursor.execute(query, args)
            for result in cursor:
                print(result)

In [18]:
db = ConnectionHelper(connection)

Note o padrão de *'dependency injection'* aqui: ao invés de criar a conexão dentro do construtor da classe ConnectionHelper, passamos um objeto 'connection' que será armazenado dentro do ConnectionHelper. Qualquer objeto 'connection' pode ser passado, de qualquer classe, desde que obedeça a interface (implícita) de um objeto de conexão ao banco de dados. Neste nosso caso, tem que ter o método `cursor()` que retorne um objeto de interação com a base de dados.

## Explorando a estrutura da base de dados

Vamos ver quais tabelas existem na base ``torneio``:

In [19]:
db("SHOW TABLES")

Executando query:
('equipe',)
('jogador',)


Para saber qual o schema da tabela `jogador`, podemos usar o comando '`DESCRIBE`'

In [20]:
db("DESCRIBE jogador")

Executando query:
('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('nome_equipe', 'varchar(80)', 'NO', 'MUL', None, '')
('nome', 'varchar(80)', 'NO', '', None, '')
('preferencia', 'int', 'NO', '', None, '')


## Consultando a base de dados

Vamos usar o comando `SELECT` para listar os conteudos da tabela 'Jogador'

In [21]:
db("SELECT * FROM jogador")

Executando query:
(2, 'Raposas Nerds', 'Raul Ayres', 0)
(3, 'Raposas Nerds', 'Luciano Hashimoto', 1)
(4, 'É nois', 'Rafael Montaigner', 2)
(5, 'É nois', 'Igor Miranda', 2)
(6, 'É nois', 'Andrew Gomes da Silva', 1)
(9, 'Raposas Nerds', 'Raul Ayres', 0)
(10, 'Raposas Nerds', 'Luciano Hashimoto', 1)
(11, 'Raposas Nerds', 'Rafael Montaigner', 2)
(12, 'É nois', 'Igor Miranda', 2)
(13, 'É nois', 'Andrew Ikeda', 1)
(14, 'É nois', 'Fábio Kurauchi', 0)


O comando acima lista todos os registros da tabela `jogador`, com todas as colunas presentes:

![Seleção da tabela inteira](imgs/tudo.png)

Vamos agora selecionar apenas algumas colunas para exibir.

In [22]:
db("SELECT nome, nome_equipe FROM jogador")

Executando query:
('Raul Ayres', 'Raposas Nerds')
('Luciano Hashimoto', 'Raposas Nerds')
('Rafael Montaigner', 'É nois')
('Igor Miranda', 'É nois')
('Andrew Gomes da Silva', 'É nois')
('Raul Ayres', 'Raposas Nerds')
('Luciano Hashimoto', 'Raposas Nerds')
('Rafael Montaigner', 'Raposas Nerds')
('Igor Miranda', 'É nois')
('Andrew Ikeda', 'É nois')
('Fábio Kurauchi', 'É nois')


Agora vemos apenas as colunas escolhidas.  A operação de seleção de colunas chama-se **projeção**.

![Projeção](imgs/projecao.png)

Vamos agora atuar na escolha de linhas, selecionando quais desejamos. Para escolher todas as linhas cujo `nome_equipe` começa com 'Rap' podemos executar a query a seguir:

In [23]:
db("SELECT * FROM jogador WHERE nome_equipe LIKE 'Rap%'")

Executando query:
(2, 'Raposas Nerds', 'Raul Ayres', 0)
(3, 'Raposas Nerds', 'Luciano Hashimoto', 1)
(9, 'Raposas Nerds', 'Raul Ayres', 0)
(10, 'Raposas Nerds', 'Luciano Hashimoto', 1)
(11, 'Raposas Nerds', 'Rafael Montaigner', 2)


Vemos apenas as linhas escolhidas. A operação de filtragem de linhas apropriadas chama-se ... **seleção**!!!

![Seleção](imgs/selecao.png)

Seleção e projeção são termos advindos da *álgebra relacional*, tópico que discutiremos em aulas futuras.

O comando `SELECT` também pode ser usado para efetuar cálculos:

In [24]:
db("SELECT 1 + 1")

Executando query:
(2,)


## Onde foram parar os dados?

MySQL mantém um grande conjunto de variáveis globais para que possamos verificar o estado do sistema. Podemos consultar essas variáveis usando SQL! Afinal, se queremos armazenar informação (neste caso, sobre o próprio sistema), nada melhor que um SGBD!

In [25]:
db("SHOW VARIABLES")

Executando query:
('activate_all_roles_on_login', 'OFF')
('admin_address', '')
('admin_port', '33062')
('admin_ssl_ca', '')
('admin_ssl_capath', '')
('admin_ssl_cert', '')
('admin_ssl_cipher', '')
('admin_ssl_crl', '')
('admin_ssl_crlpath', '')
('admin_ssl_key', '')
('admin_tls_ciphersuites', '')
('admin_tls_version', 'TLSv1.2,TLSv1.3')
('authentication_policy', '*,,')
('auto_generate_certs', 'ON')
('auto_increment_increment', '1')
('auto_increment_offset', '1')
('autocommit', 'OFF')
('automatic_sp_privileges', 'ON')
('avoid_temporal_upgrade', 'OFF')
('back_log', '151')
('basedir', 'C:\\Program Files\\MySQL\\MySQL Server 8.0\\')
('big_tables', 'OFF')
('bind_address', '*')
('binlog_cache_size', '32768')
('binlog_checksum', 'CRC32')
('binlog_direct_non_transactional_updates', 'OFF')
('binlog_encryption', 'OFF')
('binlog_error_action', 'ABORT_SERVER')
('binlog_expire_logs_auto_purge', 'ON')
('binlog_expire_logs_seconds', '2592000')
('binlog_format', 'ROW')
('binlog_group_commit_sync_delay

Para descobrir o valor de uma variável específica, use a cláusula `WHERE`:

In [26]:
db("SHOW VARIABLES WHERE Variable_name = 'version'")

Executando query:
('version', '8.0.41')


Para listar todas as variáveis com "dir" no nome:

In [27]:
db("SHOW VARIABLES WHERE Variable_name LIKE '%dir%'")

Executando query:
('basedir', 'C:\\Program Files\\MySQL\\MySQL Server 8.0\\')
('binlog_direct_non_transactional_updates', 'OFF')
('character_sets_dir', 'C:\\Program Files\\MySQL\\MySQL Server 8.0\\share\\charsets\\')
('datadir', 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Data\\')
('innodb_data_home_dir', '')
('innodb_directories', '')
('innodb_doublewrite_dir', '')
('innodb_log_group_home_dir', '.\\')
('innodb_max_dirty_pages_pct', '90.000000')
('innodb_max_dirty_pages_pct_lwm', '10.000000')
('innodb_redo_log_archive_dirs', '')
('innodb_temp_tablespaces_dir', '.\\#innodb_temp\\')
('innodb_tmpdir', '')
('innodb_undo_directory', '.\\')
('lc_messages_dir', 'C:\\Program Files\\MySQL\\MySQL Server 8.0\\share\\')
('plugin_dir', 'C:\\Program Files\\MySQL\\MySQL Server 8.0\\lib\\plugin\\')
('replica_load_tmpdir', 'C:\\WINDOWS\\SERVIC~1\\NETWOR~1\\AppData\\Local\\Temp')
('slave_load_tmpdir', 'C:\\WINDOWS\\SERVIC~1\\NETWOR~1\\AppData\\Local\\Temp')
('tmpdir', 'C:\\WINDOWS\\SERVIC~1\\NETWOR~1\\Ap

Observe a variável `datadir`: ela contém o diretório onde o MySQL armazena nossos dados. Por exemplo no Windows: `C:\ProgramData\MySQL\MySQL Server 8.0\Data\`

<center><img src='imgs/datadir.png'/></center>

Note que esse diretório também tem várias chaves criptográficas! Uma vez que o acesso físico ao seu servidor for comprometido, você pode perder completamente o controle, pois o invasor agora tem até mesmo as chaves de acesso!

Dentro do diretório `torneio` temos nossa base de dados no disco:

<center><img src='imgs/torneio.png'/></center>


## Carregando dados em massa

Se você tem um arquivo com vários itens de dados, o melhor jeito de inserí-los no banco de dados é usar os comandos de carga de dados do seu SGBD preferido. No MySQL, veja a documentação do comando `LOAD DATA` em https://dev.mysql.com/doc/refman/8.0/en/load-data.html

Daqui para frente acostume-se a consultar a documentação de todo novo comando que você descobrir!

# `UPDATE`

Vamos alterar informações na nossa base. Para isso vamos usar o comando `UPDATE`. 

Suponha que desejamos alterar o grito de guerra da equipe "Raposas Nerds" de 'sudo vencer!' para 'sudo vencer --force!':

In [28]:
db("UPDATE equipe SET grito='sudo vencer --force!' WHERE nome='Raposas Nerds'")

Executando query:


Verificando o resultado:

In [29]:
db("SELECT * from equipe")

Executando query:
('É nois', 'Olha eu mamãe!')
('Raposas Nerds', 'sudo vencer --force!')


**Atividade:** Passe o "Rafael Montaigner" para o time "É nois".

In [30]:
db("SELECT * FROM jogador")

Executando query:
(2, 'Raposas Nerds', 'Raul Ayres', 0)
(3, 'Raposas Nerds', 'Luciano Hashimoto', 1)
(4, 'É nois', 'Rafael Montaigner', 2)
(5, 'É nois', 'Igor Miranda', 2)
(6, 'É nois', 'Andrew Gomes da Silva', 1)
(9, 'Raposas Nerds', 'Raul Ayres', 0)
(10, 'Raposas Nerds', 'Luciano Hashimoto', 1)
(11, 'Raposas Nerds', 'Rafael Montaigner', 2)
(12, 'É nois', 'Igor Miranda', 2)
(13, 'É nois', 'Andrew Ikeda', 1)
(14, 'É nois', 'Fábio Kurauchi', 0)


In [31]:
# Responda aqui!
db("UPDATE jogador SET nome_equipe='É nois' where nome='Rafael Montaigner'")

Executando query:


In [32]:
db("SELECT * FROM jogador")

Executando query:
(2, 'Raposas Nerds', 'Raul Ayres', 0)
(3, 'Raposas Nerds', 'Luciano Hashimoto', 1)
(4, 'É nois', 'Rafael Montaigner', 2)
(5, 'É nois', 'Igor Miranda', 2)
(6, 'É nois', 'Andrew Gomes da Silva', 1)
(9, 'Raposas Nerds', 'Raul Ayres', 0)
(10, 'Raposas Nerds', 'Luciano Hashimoto', 1)
(11, 'É nois', 'Rafael Montaigner', 2)
(12, 'É nois', 'Igor Miranda', 2)
(13, 'É nois', 'Andrew Ikeda', 1)
(14, 'É nois', 'Fábio Kurauchi', 0)


**Atividade:** Tente mudar o nome de "Andrew Ikeda" para "Andrew Gomes da Silva". O que aconteceu?

In [33]:
# Responda aqui!
db("UPDATE jogador SET nome='Andrew Gomes da Silva' where nome='Andrew Ikeda'")

Executando query:


In [34]:
db("SELECT * FROM jogador")

Executando query:
(2, 'Raposas Nerds', 'Raul Ayres', 0)
(3, 'Raposas Nerds', 'Luciano Hashimoto', 1)
(4, 'É nois', 'Rafael Montaigner', 2)
(5, 'É nois', 'Igor Miranda', 2)
(6, 'É nois', 'Andrew Gomes da Silva', 1)
(9, 'Raposas Nerds', 'Raul Ayres', 0)
(10, 'Raposas Nerds', 'Luciano Hashimoto', 1)
(11, 'É nois', 'Rafael Montaigner', 2)
(12, 'É nois', 'Igor Miranda', 2)
(13, 'É nois', 'Andrew Gomes da Silva', 1)
(14, 'É nois', 'Fábio Kurauchi', 0)


**Atividade:** Tente agora mudar o nome do time "É nois" para "Somos nos". O que aconteceu?

In [35]:
# Responda aqui!
db("UPDATE equipe SET nome='Somos nos' where nome='É nois'")

Executando query:


In [36]:
db("SELECT * FROM equipe")

Executando query:
('Raposas Nerds', 'sudo vencer --force!')
('Somos nos', 'Olha eu mamãe!')


## `COMMIT` e `ROLLBACK`

Note que as ultimas atividades (de `UPDATE`) foram feitas sem executar `connection.commit()`. Neste caso todas as modificações realizadas ainda não foram registradas no banco de dados! Estas mudanças existem apenas na nossa *sessão*. Vamos verificar esse fenômeno. Primeiro vamos ver o estado das nossas tabelas:

In [37]:
db("SELECT * FROM jogador")

Executando query:
(2, 'Raposas Nerds', 'Raul Ayres', 0)
(3, 'Raposas Nerds', 'Luciano Hashimoto', 1)
(4, 'Somos nos', 'Rafael Montaigner', 2)
(5, 'Somos nos', 'Igor Miranda', 2)
(6, 'Somos nos', 'Andrew Gomes da Silva', 1)
(9, 'Raposas Nerds', 'Raul Ayres', 0)
(10, 'Raposas Nerds', 'Luciano Hashimoto', 1)
(11, 'Somos nos', 'Rafael Montaigner', 2)
(12, 'Somos nos', 'Igor Miranda', 2)
(13, 'Somos nos', 'Andrew Gomes da Silva', 1)
(14, 'Somos nos', 'Fábio Kurauchi', 0)


In [38]:
db("SELECT * FROM equipe")

Executando query:
('Raposas Nerds', 'sudo vencer --force!')
('Somos nos', 'Olha eu mamãe!')


Agora vamos verificar o estado do banco de dados lá no MySQL Workbench:

<center><img src='imgs/equipes.png'/></center>

<center><img src='imgs/jogadores.png'/></center>

Está diferente! Isso acontece porque as mudanças da nossa sessão ainda não foram *committed*, não foram registradas em definitivo. Para registrar as mudanças você pode executar o comando `connection.commit()`, ou equivalentemente rodar o comando SQL `COMMIT`. Se você se arrependeu das mudanças realizadas na sessão, jogue elas fora com o comando `connection.rollback()` ou equivalentemente o comando SQL `ROLLBACK`.

Vamos então concretizar nossas mudanças:

In [ ]:
connection.commit()
# ou então db("COMMIT"), é a mesma coisa.

Agora vamos verificar lá no MySQL Workbench:

<center><img src='imgs/equipes2.png'/></center>

<center><img src='imgs/jogadores2.png'/></center>

Podemos constatar que as mudanças feitas na nossa sessão agora são visíveis na outra sessão (a do MySQL Workbench). Vamos revisitar futuramente os comandos `COMMIT` e `ROLLBACK` na aula de transações SQL.

## `DELETE`

Vamos remover o 'Fábio Kurauchi' da tabela `jogador`

In [39]:
try:
    db("DELETE FROM jogador WHERE nome='Fábio Kurauchi'")
    connection.commit()
except Exception as e:
    print(f"{type(e)}: {e}")
    connection.rollback()

Executando query:


Verificando o resultado:

In [40]:
db("SELECT * FROM jogador")

Executando query:
(2, 'Raposas Nerds', 'Raul Ayres', 0)
(3, 'Raposas Nerds', 'Luciano Hashimoto', 1)
(4, 'Somos nos', 'Rafael Montaigner', 2)
(5, 'Somos nos', 'Igor Miranda', 2)
(6, 'Somos nos', 'Andrew Gomes da Silva', 1)
(9, 'Raposas Nerds', 'Raul Ayres', 0)
(10, 'Raposas Nerds', 'Luciano Hashimoto', 1)
(11, 'Somos nos', 'Rafael Montaigner', 2)
(12, 'Somos nos', 'Igor Miranda', 2)
(13, 'Somos nos', 'Andrew Gomes da Silva', 1)


**Atividade:** Remova da tabela de jogadores todos aqueles que preferem jogar cowboy.

In [ ]:
COWBOY

1

In [ ]:
# Responda aqui!
db(f"DELETE FROM jogador WHERE preferencia={COWBOY}")

Executando query:


In [ ]:
db("SELECT * FROM jogador")

Executando query:
(2, 'Raposas Nerds', 'Raul Ayres', 0)
(4, 'É nois', 'Rafael Montaigner', 2)
(5, 'É nois', 'Igor Miranda', 2)


Agora vamos tentar remover o time 'É nois':

In [ ]:
try:
    db("DELETE FROM equipe WHERE nome='É nois'")
    connection.commit()
except Exception as e:
    print(f"{type(e)}: {e}")
    connection.rollback()

Executando query:
<class 'mysql.connector.errors.IntegrityError'>: 1451 (23000): Cannot delete or update a parent row: a foreign key constraint fails (`torneio`.`jogador`, CONSTRAINT `jogador_ibfk_1` FOREIGN KEY (`nome_equipe`) REFERENCES `equipe` (`nome`))


Novamente o malfadado erro `IntegrityError`. Isso acontece porque o SGBD está fazendo exatamente o que você pediu: garantindo a integridade dos dados! Afinal, se removermos a equipe `'É nois'`, o que fazemos com as linhas da tabela-filha que referenciam esse registro?

A restrição de chave estrangeira no nome da equipe (em `script_001.sql` ela foi chamada de `fk_equipe`) especifica, implicitamente, que a ação de apagar um registro deve ser bloqueada se impactar tabelas filhas. Podemos mudar essa restrição.

## `ON UPDATE / ON DELETE`

Vamos reescrever a restrição de chave estrangeira. Copie o script a seguir para `script_002.sql`.

```mysql
USE torneio;

ALTER TABLE jogador
    DROP FOREIGN KEY fk_equipe;

ALTER TABLE jogador
    ADD CONSTRAINT fk_equipe FOREIGN KEY (nome_equipe) REFERENCES equipe (nome)
    ON DELETE CASCADE
    ON UPDATE CASCADE;

```

Feche a conexão atual com o banco de dados: ela está bloqueando alterações no schema. Não se esqueça de efetivar as modificações passadas com `commit()`, se elas não foram *committed* até o momento:

In [ ]:
connection.commit()
connection.close()

Agora rode `script_002.sql` no MySQL Workbench. Se tudo deu certo você deve observar a mudança de ação na restrição de chave estrageira:

<center><img src='imgs/cascade.png'/></center>

Reabra a conexão e recrie o *helper*

In [43]:
connection = mysql.connector.connect(**connection_options)
db = ConnectionHelper(connection)

Vamos agora tentar mudar o nome da equipe 'É nois' para 'Somos nós':

In [ ]:
db("UPDATE equipe SET nome='Somos nós' WHERE nome='É nois'")

Executando query:


Verificando o resultado:

In [ ]:
db("SELECT * from equipe")

Executando query:
('Raposas Nerds', 'sudo vencer --force!')
('Somos nós', 'Olha eu mamãe!')


Vamos verificar o que aconteceu com a tabela `jogador`

In [ ]:
db("SELECT * FROM jogador")

Executando query:
(2, 'Raposas Nerds', 'Raul Ayres', 0)
(3, 'Raposas Nerds', 'Luciano Hashimoto', 1)
(4, 'Somos nós', 'Rafael Montaigner', 2)
(5, 'Somos nós', 'Igor Miranda', 2)
(6, 'Somos nós', 'Andrew Gomes da Silva', 1)


Como você pode ver, a mudança de nome foi devidamente propagada, cortesia do `ON UPDATE CASCADE`.

Já o `ON DELETE CASCADE` é mais bruto. Vamos remover o time 'Somos nós':

In [ ]:
db("DELETE FROM equipe WHERE nome='Somos nós'")

Executando query:


In [ ]:
db("SELECT * from equipe")

Executando query:
('Raposas Nerds', 'sudo vencer --force!')


Ok. E os jogadores?

In [ ]:
db("SELECT * FROM jogador")

Executando query:
(2, 'Raposas Nerds', 'Raul Ayres', 0)
(3, 'Raposas Nerds', 'Luciano Hashimoto', 1)


A ação `CASCADE` associada ao evento `ON DELETE` acabou por limpar a tabela `jogador` tambem.

Vamos cancelar todas essas ações com o comando `rollback()`, que é o oposto de `commit()`

In [ ]:
connection.rollback()

Verificando se deu certo:

In [ ]:
db("SELECT * FROM jogador")

Executando query:
(2, 'Raposas Nerds', 'Raul Ayres', 0)
(3, 'Raposas Nerds', 'Luciano Hashimoto', 1)
(4, 'É nois', 'Rafael Montaigner', 2)
(5, 'É nois', 'Igor Miranda', 2)
(6, 'É nois', 'Andrew Gomes da Silva', 1)


**Atividade:** Alem de `CASCADE` e `RESTRICT` existem outros especificadores de ação. Ache a URL da documentação que mostra essas opções.

<div class="alert alert-success">

Sua resposta aqui! Dê dois cliques e edite.
    
    Escreva AQUI!

</div>

## `REPLACE` = `DELETE` + `INSERT`

O comando `REPLACE` lembra um pouco o comando `UPDATE`, e é aí que mora o perigo! `REPLACE` é na verdade uma combinação de `DELETE` seguido de `INSERT`. Vamos exemplificar: suponha que eu quero atualizar o grito do time 'É nois' para 'Vai dar ruim!', mas ao invés de usar o `UPDATE` resolvi usar o `REPLACE`:

In [ ]:
db("SELECT * from equipe")
db("SELECT * from jogador")

Executando query:
('É nois', 'Olha eu mamãe!')
('Raposas Nerds', 'sudo vencer --force!')
Executando query:
(2, 'Raposas Nerds', 'Raul Ayres', 0)
(3, 'Raposas Nerds', 'Luciano Hashimoto', 1)
(4, 'É nois', 'Rafael Montaigner', 2)
(5, 'É nois', 'Igor Miranda', 2)
(6, 'É nois', 'Andrew Gomes da Silva', 1)


In [ ]:
db("REPLACE INTO equipe VALUES ('É nois', 'Vai dar ruim!')")

Executando query:


In [ ]:
db("SELECT * from equipe")

Executando query:
('É nois', 'Vai dar ruim!')
('Raposas Nerds', 'sudo vencer --force!')


In [ ]:
db("SELECT * from jogador")

Executando query:
(2, 'Raposas Nerds', 'Raul Ayres', 0)
(3, 'Raposas Nerds', 'Luciano Hashimoto', 1)


Aparentemente deu ruim!

**Atividade:** Explique

<div class="alert alert-success">

Sua resposta aqui! Dê dois cliques e edite.
    
    Escreva AQUI!

</div>

Ufa, por hoje é só! Feche a conexão e até a próxima aula!

In [44]:
connection.rollback()
connection.close()

## Atividade para prática de DML

Faça a atividade para prática de DML.

### Conferindo a API de Autograding

#### Tarefas e Notas
Vamos conferir as tarefas e notas

In [45]:
ia.tasks()

|    | Atividade   | De                  | Até                 |
|---:|:------------|:--------------------|:--------------------|
|  0 | newborn     | 2025-02-01 00:00:00 | 2025-05-30 00:00:00 |
|  1 | select01    | 2025-02-05 00:00:00 | 2025-02-15 23:59:59 |
|  2 | ddl         | 2025-02-19 00:00:00 | 2025-02-26 23:59:59 |
|  3 | dml         | 2025-02-24 00:00:00 | 2025-03-09 23:59:59 |
|  4 | agg_join    | 2025-02-26 00:00:00 | 2025-03-09 23:59:59 |

In [46]:
ia.grades(task="dml")

|    | Atividade   | Exercício   |   Peso |   Nota |
|---:|:------------|:------------|-------:|-------:|
|  0 | dml         | ex01        |      1 |     10 |
|  1 | dml         | ex02        |      1 |     10 |
|  2 | dml         | ex03        |      1 |     10 |
|  3 | dml         | ex04        |      1 |     10 |
|  4 | dml         | ex05        |      1 |      0 |
|  5 | dml         | ex06        |      1 |      0 |

In [47]:
ia.grades(by="TASK")

|    | Tarefa   |   Nota |
|---:|:---------|-------:|
|  0 | agg_join |   0    |
|  1 | ddl      |  10    |
|  2 | dml      |   6.67 |
|  3 | newborn  |  10    |
|  4 | select01 |  10    |

### Criar Base de Dados

Vamos começar pela criação da base de dados. Ela deverá se chamar `vendinha`.

<img src="imgs/vendinha.png">

<div class="alert alert-info">

Execute no workbench o script `vendinha.sql` disponibilizado pelo professor.
    
</div>

In [48]:
connection_options_atv = {
    "host": os.getenv("MD_DB_SERVER"),
    "user": os.getenv("MD_DB_USERNAME"),
    "password": os.getenv("MD_DB_PASSWORD"),
    "database": "vendinha"
}

connection_atv = mysql.connector.connect(**connection_options_atv)

db_atv = ConnectionHelper(connection_atv)

**Exercício 1**: Faça uma query para inserir a região `"N"` com descrição `"Norte"` na tabela `regiao`.

<div class="alert alert-info">

**DICA:**
    
Agora que temos a função `db` para conexão com o banco, podemos testar nossas queries aqui mesmo no notebook, alternando comandos de **DML** com `SELECT` para conferir o resultado.
</div>

In [49]:
# Deixa eu ver como está
db_atv("SELECT * FROM regiao")

Executando query:


In [50]:
# Agora deixa eu responder a questão


sql_ex01 = """
INSERT INTO regiao (regiao, descricao) VALUES ('N', 'Norte')
"""

db_atv(sql_ex01)

Executando query:


In [51]:
# Deixa eu ver o RESULTADO!
db_atv("SELECT * FROM regiao")

Executando query:
('N', 'Norte')


In [ ]:
ia.sender(answer="sql_ex01", task="dml", question="ex01", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex01', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 2**: Em uma única query (sem fazer vários `INSERT`), insira as regiões na tabela `regiao`:

- a região `"NE"` com descrição `"Nordeste"`.
- a região `"SE"` com descrição `"Sudeste"`.
- a região `"S"` com descrição `"Sul"`.
- a região `"CO"` com descrição `"Centro-Oeste"`.

In [52]:
# Deixa eu ver como está
db_atv("SELECT * FROM regiao")

Executando query:
('N', 'Norte')


In [53]:
# Agora deixa eu responder a questão
sql_ex02 = """
INSERT INTO regiao (regiao, descricao) VALUES 
    ('NE', 'Nordeste'),
    ('SE', 'Sudeste'),
    ('S', 'Sul'),
    ('CO', 'Centro-Oeste');
"""

db_atv(sql_ex02)

Executando query:


In [54]:
# Deixa eu ver o RESULTADO!
db_atv("SELECT * FROM regiao")

Executando query:
('CO', 'Centro-Oeste')
('N', 'Norte')
('NE', 'Nordeste')
('S', 'Sul')
('SE', 'Sudeste')


In [ ]:
ia.sender(answer="sql_ex02", task="dml", question="ex02", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex02', style=ButtonStyle()), Output()), _dom_classes=('widget…

In [55]:
connection_atv.commit()

**Exercício 3**: Considere que as seguinte queries foram executadas:

```mysql
INSERT INTO vendinha.uf
    (uf, descricao, regiao)
VALUES
    ('sp', 'São Paulo', 'SE'),
    ('mg', 'Minas Gerais', 'SE'),
    ('pr', 'Paraná', 'S'),
    ('am', 'Amazonas', 'N'),
    ('ba', 'Bahia', 'NE');
```

**Obs**: vamos supor que as cinco regiões do BR estão cadastradas. Cadastre-as em sua base local usando a chamada do banco (aqui do notebook mesmo)!

Crie uma que altere para **maiúsculo** o `uf` apenas dos estados da região **sudeste**.

In [ ]:
db_atv("""
INSERT INTO vendinha.uf
    (uf, descricao, regiao)
VALUES
    ('sp', 'São Paulo', 'SE'),
    ('mg', 'Minas Gerais', 'SE'),
    ('pr', 'Paraná', 'S'),
    ('am', 'Amazonas', 'N'),
    ('ba', 'Bahia', 'NE');
""")

Executando query:


In [ ]:
# Deixa eu ver como está
db_atv("SELECT * FROM uf")

Executando query:
('am', 'Amazonas', 'N')
('ba', 'Bahia', 'NE')
('mg', 'Minas Gerais', 'SE')
('pr', 'Paraná', 'S')
('sp', 'São Paulo', 'SE')


In [ ]:
# Agora deixa eu responder a questão
sql_ex03 = """
UPDATE uf SET uf=UPPER(uf) WHERE regiao='SE';
"""

db_atv(sql_ex03)

Executando query:


In [ ]:
# Deixa eu ver o RESULTADO!
db_atv("SELECT * FROM uf")

Executando query:
('am', 'Amazonas', 'N')
('ba', 'Bahia', 'NE')
('MG', 'Minas Gerais', 'SE')
('pr', 'Paraná', 'S')
('SP', 'São Paulo', 'SE')


In [ ]:
ia.sender(answer="sql_ex03", task="dml", question="ex03", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex03', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 4**: Considere que as seguinte queries foram executadas:

<div class="alert alert-info">

Perceba que os `uf` estão em maúsculo neste exercício!

</div>

**Tabela `uf`**:
```mysql
INSERT INTO vendinha.uf
    (uf, descricao, regiao)
VALUES
    ('SP', 'São Paulo', 'SE'),
    ('MG', 'Minas Gerais', 'SE'),
    ('PR', 'Paraná', 'S'),
    ('AM', 'Amazonas', 'N'),
    ('BA', 'Bahia', 'NE');
```

**Tabela `cidade`**:
```mysql
INSERT INTO vendinha.cidade
    (id, descricao, uf)
VALUES
    (20, 'São Paulo', 'SP'),
    (21, 'Campinas', 'SP'),
    (22, 'Salvador', 'BA'),
    (23, 'Manaus', 'AM'),
    (24, 'Belo Horizonte', 'MG'),
    (25, 'São Roque de Minas', 'MG');
```

**Tabela `vendedor`**:
```mysql
INSERT INTO vendinha.vendedor
    (id, nome, data_nasc, data_cad, ativo)
VALUES
    (100, 'Maria Roque', '1988-01-01', '2023-06-11', 1),
    (101, 'Ana Benedita', '1970-12-09', '2023-07-15', 1),
    (102, 'Silvio Jardim', '1988-12-25', '2023-08-01', 1),
    (103, 'Bruna Fontana', '1981-07-05', '2023-09-01', 1),
    (104, 'Tulio Maravilha', '1978-09-22', '2023-08-07', 1),
    (105, 'Gino Pereira', '1964-04-03', '2023-08-25', 0),
    (106, 'Camila Oliveira', '1992-08-05', '2023-09-01', 1),
    (107, 'Mariana Souza', '1985-08-29', '2023-09-01', 1);
```


**Obs**: vamos supor que as cinco regiões do BR estão cadastradas. Cadastre-as em sua base local usando a chamada do banco (aqui do notebook mesmo)!

Foi descoberto que todos os vendedores cadastrados em **agosto de 2023** são bots e devem ser removidos da base. Construa sua query!

In [ ]:
connection_atv.rollback()

In [ ]:
# Deixa eu ver como está
db_atv("SELECT * FROM vendedor")

Executando query:
(100, 'Maria Roque', datetime.date(1988, 1, 1), datetime.date(2023, 6, 11), 1)
(101, 'Ana Benedita', datetime.date(1970, 12, 9), datetime.date(2023, 7, 15), 1)
(102, 'Silvio Jardim', datetime.date(1988, 12, 25), datetime.date(2023, 8, 1), 1)
(103, 'Bruna Fontana', datetime.date(1981, 7, 5), datetime.date(2023, 9, 1), 1)
(104, 'Tulio Maravilha', datetime.date(1978, 9, 22), datetime.date(2023, 8, 7), 1)
(105, 'Gino Pereira', datetime.date(1964, 4, 3), datetime.date(2023, 8, 25), 0)
(106, 'Camila Oliveira', datetime.date(1992, 8, 5), datetime.date(2023, 9, 1), 1)
(107, 'Mariana Souza', datetime.date(1985, 8, 29), datetime.date(2023, 9, 1), 1)


In [ ]:
# Agora deixa eu responder a questão
sql_ex04 = """
DELETE FROM vendedor WHERE data_cad BETWEEN '2023-8-1' and '2023-8-31';
"""

db_atv(sql_ex04)

Executando query:


In [ ]:
# Deixa eu ver o RESULTADO!
db_atv("SELECT * FROM vendedor")

Executando query:
(100, 'Maria Roque', datetime.date(1988, 1, 1), datetime.date(2023, 6, 11), 1)
(101, 'Ana Benedita', datetime.date(1970, 12, 9), datetime.date(2023, 7, 15), 1)
(103, 'Bruna Fontana', datetime.date(1981, 7, 5), datetime.date(2023, 9, 1), 1)
(106, 'Camila Oliveira', datetime.date(1992, 8, 5), datetime.date(2023, 9, 1), 1)
(107, 'Mariana Souza', datetime.date(1985, 8, 29), datetime.date(2023, 9, 1), 1)


In [ ]:
ia.sender(answer="sql_ex04", task="dml", question="ex04", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex04', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 5**: Considere que as seguinte queries foram executadas:

<div class="alert alert-info">

Perceba que os `uf` estão em maúsculo neste exercício!

</div>

**Tabela `uf`**:
```mysql
INSERT INTO vendinha.uf
    (uf, descricao, regiao)
VALUES
    ('SP', 'São Paulo', 'SE'),
    ('MG', 'Minas Gerais', 'SE'),
    ('PR', 'Paraná', 'S'),
    ('AM', 'Amazonas', 'N'),
    ('BA', 'Bahia', 'NE');
```

**Tabela `cidade`**:
```mysql
INSERT INTO vendinha.cidade
    (id, descricao, uf)
VALUES
    (20, 'São Paulo', 'SP'),
    (21, 'Campinas', 'SP'),
    (22, 'Salvador', 'BA'),
    (23, 'Manaus', 'AM'),
    (24, 'Belo Horizonte', 'MG'),
    (25, 'São Roque de Minas', 'MG');
```

**Tabela `vendedor`**:
```mysql
INSERT INTO vendinha.vendedor
    (id, nome, data_nasc, data_cad, ativo)
VALUES
    (100, 'Maria Roque', '1988-01-01', '2023-06-11', 1),
    (101, 'Ana Benedita', '1970-12-09', '2023-07-15', 1),
    (102, 'Silvio Jardim', '1988-12-25', '2023-08-01', 1),
    (103, 'Bruna Fontana', '1981-07-05', '2023-09-01', 1),
    (104, 'Tulio Maravilha', '1978-09-22', '2023-08-07', 1),
    (105, 'Gino Pereira', '1964-04-03', '2023-08-25', 0),
    (106, 'Camila Oliveira', '1992-08-05', '2023-09-01', 1),
    (107, 'Mariana Souza', '1985-08-29', '2023-09-01', 1);
```


**Obs**: vamos supor que as cinco regiões do BR estão cadastradas. Cadastre-as em sua base local usando a chamada do banco (aqui do notebook mesmo)!

Dadas as relações de **vendedores** e **cidades onde vendedor vende**:

- Bruna Fontana:
    - São Paulo SP
    - Campinas SP
- Silvio Jardim:
    - São Paulo SP
    - Belo Horizonte MG
    - São Roque de Minas
- Camila Oliveira:
    - Manaus AM
    
Construa uma query para popular a tabela `vendedor_vende_cidade`.

In [56]:
db_atv("SELECT * FROM uf")

Executando query:
('AM', 'Amazonas', 'N')
('BA', 'Bahia', 'NE')
('MG', 'Minas Gerais', 'SE')
('PR', 'Paraná', 'S')
('SP', 'São Paulo', 'SE')


In [57]:
# Deixa eu ver como está
db_atv("SELECT * FROM vendedor_vende_cidade")

Executando query:


In [59]:
# Agora deixa eu responder a questão
sql_ex05 = """
INSERT INTO vendinha.vendedor_vende_cidade
    (id_vendedor, id_cidade)
VALUES
    (103, 20),
    (103, 21),
    (102, 20),
    (102, 24),
    (102, 25),
    (106, 23);
"""

db_atv(sql_ex05)

Executando query:


In [60]:
# Deixa eu ver o RESULTADO!
db_atv("SELECT * FROM vendedor_vende_cidade")

Executando query:
(102, 20)
(103, 20)
(103, 21)
(106, 23)
(102, 24)
(102, 25)


In [61]:
ia.sender(answer="sql_ex05", task="dml", question="ex05", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex05', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 6**: Considere que as seguinte queries foram executadas:

**Tabela `uf`**:
```mysql
INSERT INTO vendinha.uf
    (uf, descricao, regiao)
VALUES
    ('SP', 'São Paulo', 'SE'),
    ('MG', 'Minas Gerais', 'SE'),
    ('PR', 'Paraná', 'S'),
    ('AM', 'Amazonas', 'N'),
    ('BA', 'Bahia', 'NE');
```

**Tabela `cidade`**:
```mysql
INSERT INTO vendinha.cidade
    (id, descricao, uf)
VALUES
    (20, 'São Paulo', 'SP'),
    (21, 'Campinas', 'SP'),
    (22, 'Salvador', 'BA'),
    (23, 'Manaus', 'AM'),
    (24, 'Belo Horizonte', 'MG'),
    (25, 'São Roque de Minas', 'MG');
```

**Tabela `vendedor`**:
```mysql
INSERT INTO vendinha.vendedor
    (id, nome, data_nasc, data_cad, ativo)
VALUES
    (100, 'Maria Roque', '1988-01-01', '2023-06-11', 1),
    (101, 'Ana Benedita', '1970-12-09', '2023-07-15', 1),
    (102, 'Silvio Jardim', '1988-12-25', '2023-08-01', 1),
    (103, 'Bruna Fontana', '1981-07-05', '2023-09-01', 1),
    (104, 'Tulio Maravilha', '1978-09-22', '2023-08-07', 1),
    (105, 'Gino Pereira', '1964-04-03', '2023-08-25', 0),
    (106, 'Camila Oliveira', '1992-08-05', '2023-09-01', 1),
    (107, 'Mariana Souza', '1985-08-29', '2023-09-01', 1);
```

Ainda, considere uma nova tabela chamada `vendedor_ativo_dia` com o esquema:

| Coluna         | Tipo         | PK (Primary Key?) | Not Null? | Autoinc?
|----------------|--------------|-------------------|-----------|-----------|
| id             | INT          |   True            |    True   |     True  |
| id_vendedor    | INT          |                   |    True   |           |
| ativo          | TYNYINT      |                   |    True   |           |
| data_registro  | DATE         |                   |    True   |           |

Considere que a tabela `vendedor_ativo_dia` é utilizada para criar um registro histórico de todos os vendedores que estão ativos em cada data.

**Obs**: vamos supor que as cinco regiões do BR estão cadastradas. Cadastre-as em sua base local usando a chamada do banco (aqui do notebook mesmo)!

Construa uma query que faça a inserção dos dados na tabela `vendedor_ativo_dia` a partir de um `SELECT` dos vendedores ativos da tabela de vendedor.

<div class="alert alert-danger">

Não deixe a data **hardcoded**. Pesquise como recuperar a data atual do sistema em uma query.

</div>
  
<div class="alert alert-info">

Por simplificação, não é necessário criar as constraints de chave extrangeira na tabela `vendedor_ativo_dia`.

</div>

Sugestão de resolução:
- Criar DDL para `vendedor_ativo_dia`
- Criar SELECT para pegar:
    - Vendedores ativos
    - com o dia de hoje
- Inserir estes dados (usando `SELECT`) na tabela `vendedor_ativo_dia`.

Não é necessário enviar a **DDL** para o servidor.

In [67]:
connection_atv.reconnect()

In [70]:
# Deixa eu ver como está
db_atv("SELECT * FROM vendedor WHERE ativo=1")

Executando query:
(100, 'Maria Roque', datetime.date(1988, 1, 1), datetime.date(2023, 6, 11), 1)
(101, 'Ana Benedita', datetime.date(1970, 12, 9), datetime.date(2023, 7, 15), 1)
(102, 'Silvio Jardim', datetime.date(1988, 12, 25), datetime.date(2023, 8, 1), 1)
(103, 'Bruna Fontana', datetime.date(1981, 7, 5), datetime.date(2023, 9, 1), 1)
(104, 'Tulio Maravilha', datetime.date(1978, 9, 22), datetime.date(2023, 8, 7), 1)
(106, 'Camila Oliveira', datetime.date(1992, 8, 5), datetime.date(2023, 9, 1), 1)
(107, 'Mariana Souza', datetime.date(1985, 8, 29), datetime.date(2023, 9, 1), 1)


In [73]:
# Agora deixa eu responder a questão
sql_ex06 = """
INSERT INTO vendinha.vendedor_ativo_dia
    (id_vendedor, ativo, data_registro)
SELECT
    id, ativo, CURRENT_DATE
FROM
    vendedor
WHERE
    ativo = 1;
"""

db_atv(sql_ex06)

Executando query:


In [74]:
# Deixa eu ver o RESULTADO!
db_atv("SELECT * FROM vendedor_ativo_dia")

Executando query:
(1, 100, 1, datetime.date(2025, 3, 9))
(2, 101, 1, datetime.date(2025, 3, 9))
(3, 102, 1, datetime.date(2025, 3, 9))
(4, 103, 1, datetime.date(2025, 3, 9))
(5, 104, 1, datetime.date(2025, 3, 9))
(6, 106, 1, datetime.date(2025, 3, 9))
(7, 107, 1, datetime.date(2025, 3, 9))


In [75]:
ia.sender(answer="sql_ex06", task="dml", question="ex06", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex06', style=ButtonStyle()), Output()), _dom_classes=('widget…

### Conferir Notas

Confira se as notas na atividade são as esperadas!

Primeiro na atividade atual!

In [76]:
ia.grades(by="TASK", task="dml")

|    | Tarefa   |   Nota |
|---:|:---------|-------:|
|  0 | dml      |     10 |

In [77]:
ia.grades(task="dml")

|    | Atividade   | Exercício   |   Peso |   Nota |
|---:|:------------|:------------|-------:|-------:|
|  0 | dml         | ex01        |      1 |     10 |
|  1 | dml         | ex02        |      1 |     10 |
|  2 | dml         | ex03        |      1 |     10 |
|  3 | dml         | ex04        |      1 |     10 |
|  4 | dml         | ex05        |      1 |     10 |
|  5 | dml         | ex06        |      1 |     10 |

E agora em todas as demais!

In [78]:
ia.grades(by="TASK")

|    | Tarefa   |   Nota |
|---:|:---------|-------:|
|  0 | agg_join |      0 |
|  1 | ddl      |     10 |
|  2 | dml      |     10 |
|  3 | newborn  |     10 |
|  4 | select01 |     10 |

In [79]:
ia.grades()

|    | Atividade   | Exercício   |   Peso |   Nota |
|---:|:------------|:------------|-------:|-------:|
|  0 | agg_join    | ex01        |      1 |      0 |
|  1 | agg_join    | ex02        |      1 |      0 |
|  2 | agg_join    | ex03        |      1 |      0 |
|  3 | agg_join    | ex04        |      1 |      0 |
|  4 | agg_join    | ex05        |      1 |      0 |
|  5 | agg_join    | ex06        |      1 |      0 |
|  6 | ddl         | ex02        |      1 |     10 |
|  7 | ddl         | ex03        |      1 |     10 |
|  8 | ddl         | ex04        |      1 |     10 |
|  9 | ddl         | ex05        |      1 |     10 |
| 10 | ddl         | ex06        |      1 |     10 |
| 11 | ddl         | ex07        |      1 |     10 |
| 12 | ddl         | ex09        |      1 |     10 |
| 13 | ddl         | ex10        |      1 |     10 |
| 14 | ddl         | ex11        |      1 |     10 |
| 15 | dml         | ex01        |      1 |     10 |
| 16 | dml         | ex02        |      1 |     10 |
| 17 | dml         | ex03        |      1 |     10 |
| 18 | dml         | ex04        |      1 |     10 |
| 19 | dml         | ex05        |      1 |     10 |
| 20 | dml         | ex06        |      1 |     10 |
| 21 | newborn     | ex01        |      1 |     10 |
| 22 | select01    | ex01        |      1 |     10 |
| 23 | select01    | ex02        |      1 |     10 |
| 24 | select01    | ex03        |      1 |     10 |
| 25 | select01    | ex04        |      1 |     10 |
| 26 | select01    | ex05        |      1 |     10 |

In [80]:
connection_atv.commit()
connection_atv.close()